# Popularity in twitter

In this project, I will investigate what factors can increase a person's chances of being popular on Twitter. For this project, the popularity is associated with the user's number of followers. The first thing to do is import the necessary libraries and the dataset. After that, I will investigate the different columns of this dataset and some basic statistics for the numerical features.

In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, matthews_corrcoef, confusion_matrix,make_scorer
from sklearn.model_selection import GridSearchCV
pd.set_option('display.max_columns', None)

In [3]:
random_tweets = pd.read_json("random_tweets.json",lines=True)
random_tweets.head()

,created_at,id,id_str,text,truncated,entities,metadata,source,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,in_reply_to_user_id_str,in_reply_to_screen_name,user,geo,coordinates,place,contributors,retweeted_status,is_quote_status,retweet_count,favorite_count,favorited,retweeted,lang,possibly_sensitive,quoted_status_id,quoted_status_id_str,extended_entities,quoted_status,withheld_in_countries
0,2018-07-31 13:34:40+00:00,1024287229525598210,1024287229525598208,RT @KWWLStormTrack7: We are more than a month ...,False,"{'hashtags': [], 'symbols': [], 'user_mentions...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""http://twitter.com/download/android"" ...",NaN,NaN,NaN,NaN,None,"{'id': 145388018, 'id_str': '145388018', 'name...",None,None,None,NaN,{'created_at': 'Mon Jul 30 16:49:41 +0000 2018...,False,3,0,False,False,en,NaN,NaN,NaN,NaN,NaN,NaN
1,2018-07-31 13:34:40+00:00,1024287229512953856,1024287229512953856,@hail_ee23 Thanks love its just the feeling of...,False,"{'hashtags': [], 'symbols': [], 'user_mentions...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""http://twitter.com/download/iphone"" r...",1.024128e+18,1.024128e+18,9.271857e+17,9.271857e+17,hail_ee23,"{'id': 2407992339, 'id_str': '2407992339', 'na...",None,None,"{'id': '26b41b13d49ea2bf', 'url': 'https://api...",NaN,NaN,False,0,0,False,False,en,NaN,NaN,NaN,NaN,NaN,NaN
2,2018-07-31 13:34:40+00:00,1024287229504569344,1024287229504569344,RT @TransMediaWatch: Pink News has more on the...,False,"{'hashtags': [], 'symbols': [], 'user_mentions...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""http://twitter.com/download/iphone"" r...",NaN,NaN,NaN,NaN,None,"{'id': 901579698223931392, 'id_str': '90157969...",None,None,None,NaN,{'created_at': 'Tue Jul 31 11:03:32 +0000 2018...,False,5,0,False,False,en,0.0,NaN,NaN,NaN,NaN,NaN
3,2018-07-31 13:34:40+00:00,1024287229496029190,1024287229496029184,RT @realDonaldTrump: One of the reasons we nee...,False,"{'hashtags': [], 'symbols': [], 'user_mentions...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""http://twitter.com/download/android"" ...",NaN,NaN,NaN,NaN,None,"{'id': 23438548, 'id_str': '23438548', 'name':...",None,None,None,NaN,{'created_at': 'Tue Jul 31 11:00:41 +0000 2018...,False,11106,0,False,False,en,NaN,NaN,NaN,NaN,NaN,NaN
4,2018-07-31 13:34:40+00:00,1024287229492031490,1024287229492031488,RT @First5App: This hearing of His Word doesn’...,False,"{'hashtags': [], 'symbols': [], 'user_mentions...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""http://twitter.com/download/iphone"" r...",NaN,NaN,NaN,NaN,None,"{'id': 942424782, 'id_str': '942424782', 'name...",None,None,None,NaN,{'created_at': 'Tue Jul 31 12:30:05 +0000 2018...,False,6,0,False,False,en,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
random_tweets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11099 entries, 0 to 11098
Data columns (total 31 columns):
 #   Column                     Non-Null Count  Dtype              
---  ------                     --------------  -----              
 0   created_at                 11099 non-null  datetime64[ns, UTC]
 1   id                         11099 non-null  int64              
 2   id_str                     11099 non-null  int64              
 3   text                       11099 non-null  object             
 4   truncated                  11099 non-null  bool               
 5   entities                   11099 non-null  object             
 6   metadata                   11099 non-null  object             
 7   source                     11099 non-null  object             
 8   in_reply_to_status_id      1402 non-null   float64            
 9   in_reply_to_status_id_str  1402 non-null   float64            
 10  in_reply_to_user_id        1503 non-null   float64            
 11  in

In [5]:
random_tweets.describe()

,id,id_str,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,in_reply_to_user_id_str,contributors,retweet_count,favorite_count,possibly_sensitive,quoted_status_id,quoted_status_id_str
count,1.109900e+04,1.109900e+04,1.402000e+03,1.402000e+03,1.503000e+03,1.503000e+03,0.0,11099.000000,11099.000000,3192.000000,1.154000e+03,1.154000e+03
mean,1.024287e+18,1.024287e+18,1.023683e+18,1.023683e+18,2.033213e+17,2.033213e+17,NaN,2777.956392,0.039463,0.021617,1.011126e+18,1.011126e+18
std,3.329026e+10,3.329026e+10,1.325565e+16,1.325565e+16,3.746776e+17,3.746776e+17,NaN,12180.169923,1.013902,0.145451,5.505330e+16,5.505330e+16
min,1.024287e+18,1.024287e+18,5.372616e+17,5.372616e+17,4.106300e+04,4.106300e+04,NaN,0.000000,0.000000,0.000000,6.715966e+17,6.715966e+17
25%,1.024287e+18,1.024287e+18,1.024219e+18,1.024219e+18,6.303705e+07,6.303705e+07,NaN,0.000000,0.000000,0.000000,1.023541e+18,1.023541e+18
50%,1.024287e+18,1.024287e+18,1.024279e+18,1.024279e+18,5.847539e+08,5.847539e+08,NaN,13.000000,0.000000,0.000000,1.024033e+18,1.024033e+18
75%,1.024287e+18,1.024287e+18,1.024286e+18,1.024286e+18,4.322451e+09,4.322451e+09,NaN,428.500000,0.000000,0.000000,1.024249e+18,1.024249e+18
max,1.024287e+18,1.024287e+18,1.024287e+18,1.024287e+18,1.024285e+18,1.024285e+18,NaN,413719.000000,101.000000,1.000000,1.024287e+18,1.024287e+18


# Feature Selection

To select the features, I will create a data frame called `user_metada_df` to save each user's features. Then, I will sort the data frame according to each user's follower count. 

In [6]:
random_tweets['id_user'] = random_tweets['user'].apply(lambda user: user['id'])
user = random_tweets['user'].iloc[0]
user_metadata = {key:[] for key in user.keys()}
keys = user.keys()

for i in random_tweets.index:
    user = random_tweets['user'][random_tweets['id_user']==random_tweets['id_user'].iloc[i]].values[0]
    for key in keys:
        try:
            user_metadata[key].append(user[key])
        except:
            user_metadata[key].append(None)
user_metadata_df = pd.DataFrame(user_metadata)
user_metadata_df_sort = user_metadata_df.sort_values(by="followers_count",ascending=False).reset_index()

In [7]:
user_metadata_df_sort.head(10)

,index,id,id_str,name,screen_name,location,description,url,entities,protected,followers_count,friends_count,listed_count,created_at,favourites_count,utc_offset,time_zone,geo_enabled,verified,statuses_count,lang,contributors_enabled,is_translator,is_translation_enabled,profile_background_color,profile_background_image_url,profile_background_image_url_https,profile_background_tile,profile_image_url,profile_image_url_https,profile_banner_url,profile_link_color,profile_sidebar_border_color,profile_sidebar_fill_color,profile_text_color,profile_use_background_image,has_extended_profile,default_profile,default_profile_image,following,follow_request_sent,notifications,translator_type
0,3802,20536157,20536157,Google,Google,"Mountain View, CA",News and updates from Google,https://t.co/cHZdP4uOdu,{'url': {'urls': [{'url': 'https://t.co/cHZdP4...,False,20211862,219,4,Tue Feb 10 19:14:39 +0000 2009,1277,None,None,True,True,86703,en,False,False,False,FFFFFF,http://abs.twimg.com/images/themes/theme1/bg.png,https://abs.twimg.com/images/themes/theme1/bg.png,True,http://pbs.twimg.com/profile_images/9721548722...,https://pbs.twimg.com/profile_images/972154872...,https://pbs.twimg.com/profile_banners/20536157...,0000CC,FFFFFF,EBEFF9,000000,True,False,False,False,False,False,False,regular
1,3817,2251588934,2251588934,State Bank of India,TheOfficialSBI,,Official SBI. Product|Service updates & tips. ...,https://t.co/vLRZpkxDoe,{'url': {'urls': [{'url': 'https://t.co/vLRZpk...,False,3446156,20,616,Wed Dec 18 07:27:24 +0000 2013,646,None,None,True,True,257013,en,False,False,False,C0DEED,http://abs.twimg.com/images/themes/theme1/bg.png,https://abs.twimg.com/images/themes/theme1/bg.png,False,http://pbs.twimg.com/profile_images/8749745969...,https://pbs.twimg.com/profile_images/874974596...,https://pbs.twimg.com/profile_banners/22515889...,0084B4,FFFFFF,DDEEF6,333333,True,False,False,False,False,False,False,none
2,1562,59553554,59553554,Wendy's,Wendys,,We like our tweets the same way we like to mak...,https://t.co/RfHBoUe9Qq,{'url': {'urls': [{'url': 'https://t.co/RfHBoU...,False,2747888,1264,5009,Thu Jul 23 18:54:01 +0000 2009,29670,None,None,True,True,135267,en,False,False,False,E0E0E2,http://abs.twimg.com/images/themes/theme1/bg.png,https://abs.twimg.com/images/themes/theme1/bg.png,True,http://pbs.twimg.com/profile_images/1019447050...,https://pbs.twimg.com/profile_images/101944705...,https://pbs.twimg.com/profile_banners/59553554...,CC0808,FFFFFF,E0E0E0,333333,True,False,False,False,False,False,False,none
3,2059,28515858,28515858,Atul Khatri,one_by_two,Mumbai,"A henpecked husband, a father, a comic & a wri...",https://t.co/Jd7gMlm2os,{'url': {'urls': [{'url': 'https://t.co/Jd7gMl...,False,2704420,468,269,Fri Apr 03 06:16:08 +0000 2009,3187,None,None,True,True,19663,en,False,False,False,C0DEED,http://abs.twimg.com/images/themes/theme1/bg.png,https://abs.twimg.com/images/themes/theme1/bg.png,False,http://pbs.twimg.com/profile_images/3788000005...,https://pbs.twimg.com/profile_images/378800000...,https://pbs.twimg.com/profile_banners/28515858...,1DA1F2,C0DEED,DDEEF6,333333,True,True,True,False,False,False,False,none
4,9939,15438913,15438913,Daily Mail Online,MailOnline,,For the latest updates on breaking news visit ...,https://t.co/fT7KB2eXic,{'url': {'urls': [{'url': 'https://t.co/fT7KB2...,False,2239724,2377,12895,Tue Jul 15 10:17:46 +0000 2008,694,None,None,False,True,242816,en,False,False,True,000000,http://abs.twimg.com/images/themes/theme1/bg.png,https://abs.twimg.com/images/themes/theme1/bg.png,False,http://pbs.twimg.com/profile_images/1009374243...,https://pbs.twimg.com/profile_images/100937424...,https://pbs.twimg.com/profile_banners/15438913...,004EB3,FFFFFF,E3E3E3,333333,True,False,False,False,False,False,False,regular
5,2113,256495314,256495314,Firstpost,firstpost,"Mumbai, India",https://t.co/LGesb4pt66 for India & Indians in...,http://t.co/ZhcnfIjiDt,{'url': {'urls': [{'url': 'http://t.co/ZhcnfIj...,False,1961176,1022,3910,Wed Feb 23 13:09:

In [8]:
user_metadata_df_sort.tail(10)

,index,id,id_str,name,screen_name,location,description,url,entities,protected,followers_count,friends_count,listed_count,created_at,favourites_count,utc_offset,time_zone,geo_enabled,verified,statuses_count,lang,contributors_enabled,is_translator,is_translation_enabled,profile_background_color,profile_background_image_url,profile_background_image_url_https,profile_background_tile,profile_image_url,profile_image_url_https,profile_banner_url,profile_link_color,profile_sidebar_border_color,profile_sidebar_fill_color,profile_text_color,profile_use_background_image,has_extended_profile,default_profile,default_profile_image,following,follow_request_sent,notifications,translator_type
11089,476,990236966418501635,990236966418501635,낙원,makeparadiseee,,구독,None,{'description': {'urls': []}},False,0,311,0,Sat Apr 28 14:31:05 +0000 2018,435,None,None,False,False,2423,ko,False,False,False,F5F8FA,None,None,False,http://pbs.twimg.com/profile_images/9902381794...,https://pbs.twimg.com/profile_images/990238179...,https://pbs.twimg.com/profile_banners/99023696...,1DA1F2,C0DEED,DDEEF6,333333,True,False,True,False,False,False,False,none
11090,660,1024285265559339008,1024285265559339008,rocio chen,rociochen7,,,None,{'description': {'urls': []}},False,0,0,0,Tue Jul 31 13:26:52 +0000 2018,0,None,None,False,False,2,en,False,False,False,F5F8FA,None,None,False,http://pbs.twimg.com/profile_images/1024286398...,https://pbs.twimg.com/profile_images/102428639...,None,1DA1F2,C0DEED,DDEEF6,333333,True,False,True,False,False,False,False,none
11091,7582,1024283250703753218,1024283250703753218,Shivam[Gamers Club],shivam_club,India,Shîvam kumar,None,{'description': {'urls': []}},False,0,6,0,Tue Jul 31 13:18:52 +0000 2018,2,None,None,False,False,3,en,False,False,False,F5F8FA,None,None,False,http://pbs.twimg.com/profile_images/1024284816...,https://pbs.twimg.com/profile_images/102428481...,https://pbs.twimg.com/profile_banners/10242832...,1DA1F2,C0DEED,DDEEF6,333333,True,False,True,False,False,False,False,none
11092,286,1024284507543949312,1024284507543949312,lexa [jeongcheol soulmates au],ohmymoonwalker,,,None,{'description': {'urls': []}},False,0,4,0,Tue Jul 31 13:23:51 +0000 2018,0,None,None,False,False,16,en,False,False,False,F5F8FA,None,None,False,http://pbs.twimg.com/profile_images/1024284821...,https://pbs.twimg.com/profile_images/102428482...,https://pbs.twimg.com/profile_banners/10242845...,1DA1F2,C0DEED,DDEEF6,333333,True,False,True,False,False,False,False,none
11093,1525,168591490,168591490,sonicyouth,sonicyouthhh,,,None,{'description': {'urls': []}},False,0,221,0,Tue Jul 20 09:00:46 +0000 2010,1024,None,None,False,False,5605,ko,False,False,False,C0DEED,http://abs.twimg.com/images/themes/theme1/bg.png,https://abs.twimg.com/images/themes/theme1/bg.png,False,http://pbs.twimg.com/profile_images/1010826500...,https://pbs.twimg.com/profile_images/101082650...,None,1DA1F2,C0DEED,DDEEF6,333333,True,False,True,False,False,False,False,none
11094,7962,1015569056772968448,1015569056772968448,Upraveenkumar,Upravee57916825,Udayagiri,My dad is my hero,None,{'description': {'urls': []}},False,0,19,0,Sat Jul 07 12:11:46 +0000 2018,10,None,None,False,False,11,en,False,False,False,F5F8FA,None,None,False,http://pbs.twimg.com/profile_images/1015570512...,https://pbs.twimg.com/profile_images/101557051...,https://pbs.twimg.com/profile_banners/10155690...,1DA1F2,C0DEED,DDEEF6,333333,True,True,True,False,False,False,False,none
11095,37,1024284507543949312,1024284507543949312,lexa [jeongcheol soulmates au],ohmymoonwalker,,,None,{'description': {'urls': []}},False,0,4,0,Tue Jul 31 13:23:51 +0000 2018,0,None,None,False,False,16,en,False,False,False,F5F8FA,None,None,False,http://pbs.twimg.com/profile_images/1024284821...,https://pbs.twimg.com/profile_images/102428482...,https://pbs.twimg.com/profile_banners/10242845...,1DA1F2,C0DEED,DDEEF6,333333,True,False,True,False,False,False,False,none
11096,8493,841643130743058432,841643130743058432,John Rose Jr,JohnRoseJr1,"Dunkirk, NY","46yo

Based on the first dataframe (the ten most popular) and the second dataframe (the ten most unpopular), I will use the `'geo_enabled'` (if the geolocation is enabled or not), `'verified'` (if the profile of the user is verified or not), `'default_profile'` (if the profile has a default profile), `'entities'` (mainly I will focus if there is any entity or not) and `'statuses_count'` (number of statuses), from the `user` metadata. I chose these features because they seem to have some influence on having a popular profile or not. For example, most of the top 10 most popular users have geolocalization enabled, but the most unpopular don't. Also, the people with the most statuses have many followers. The next step is to replace the `True` and `False` values with `1` and `0` labels for the first three columns. 

In [9]:
user_metadata_df['geo_enabled'] = user_metadata_df['geo_enabled'].apply(lambda user: 1 if user else 0)
user_metadata_df['verified'] = user_metadata_df['verified'].apply(lambda user: 1 if user else 0)
user_metadata_df['default_profile'] = user_metadata_df['default_profile'].apply(lambda user: 1 if user else 0)

For the column `'entities'`, if there is no key named `'url'`, it will be labeled as `0`; otherwise, the label will be `1`. This classification will be saved in a column named `'is_url_entities'`. 

In [10]:
user_metadata_df['is_url_entities'] = user_metadata_df['entities'].apply(lambda entity: 1 if 'url' in entity else 0)

# Defining popularity

To define popularity, I will calculate some basic statics to describe a popular user.

In [11]:
user_metadata_df['followers_count'].describe()

count    1.109900e+04
mean     6.009167e+03
std      2.013144e+05
min      0.000000e+00
25%      1.310000e+02
50%      4.030000e+02
75%      1.249000e+03
max      2.021186e+07
Name: followers_count, dtype: float64

Based on these statistics and to make sure that the two classes that I will create (`popular` and `nonpopular`) are balanced, I will choose the following criteria:

* If a user has more than `500` followers, it will considered to be `popular` (the label is `1`)
* If it has less than `500` followers, it will be considered to be `nonpopular` (the label is `0`)

In [12]:
user_metadata_df['popularity'] = user_metadata_df['followers_count'].apply(lambda count: 1 if count > 500 else 0)

In [13]:
train_features = user_metadata_df[['geo_enabled','verified','default_profile','is_url_entities','statuses_count','popularity']]
train_features.head()

,geo_enabled,verified,default_profile,is_url_entities,statuses_count,popularity
0,1,0,0,0,4475,0
1,1,0,0,1,3922,0
2,0,0,1,0,11546,0
3,0,0,0,0,26609,1
4,1,0,0,0,519,0


# Create the training set and the test set

Now that I have selected all the features, I will first create a data frame with the features I will use and another data frame with the labels. Then, I will split each data frame into the training and test sets. I also will scale the column `'statuses_count'`.

In [34]:
features = train_features.drop(labels='popularity',axis=1)
labels = train_features['popularity']

In [35]:
train_data,test_data,train_labels,test_labels = train_test_split(features,labels,train_size=0.8,test_size=0.2,random_state=42)

In [36]:
scaler = StandardScaler()
train_data['statuses_count'] = scaler.fit_transform(train_data['statuses_count'].values.reshape(-1,1))
test_data['statuses_count'] = scaler.transform(test_data['statuses_count'].values.reshape(-1,1))

# Using a classifier

After the feature selection and preparation of the data, we are now ready to fit this data into a classifier. I will use two classifiers in this project, `Decision Tree` and `Random Forest`. To evaluate the performance of each model, I created a function called `evaluate_model` that will print the results of the most common metrics for binary classification. 

In [19]:
def evaluate_model(labels,predict):
    accuracy = accuracy_score(labels,predict)
    precision = precision_score(labels,predict)
    recall = recall_score(labels,predict)
    f1 = f1_score(labels,predict)
    mcc = matthews_corrcoef(labels,predict)
    confusion = confusion_matrix(labels,predict)
    print("Accuracy:", accuracy)
    print("Precision:", precision)
    print("Recall:", recall)
    print("F1-score:", f1)
    print("MCC:", mcc)
    print("Confusion matrix:", confusion)


## Decision Tree

In [20]:
model = DecisionTreeClassifier(random_state=42)
model.fit(train_data,train_labels)
predict = model.predict(test_data)
evaluate_model(test_labels,predict)

Accuracy: 0.6927927927927928
Precision: 0.6639260020554985
Recall: 0.6453546453546454
F1-score: 0.6545086119554204
MCC: 0.37817085591872796
Confusion matrix: [[892 327]
 [355 646]]


## Random Forest

In [21]:
model = RandomForestClassifier(random_state=42,n_jobs=-1)
model.fit(train_data,train_labels)
predict = model.predict(test_data)
evaluate_model(test_labels,predict)

Accuracy: 0.7004504504504504
Precision: 0.6761006289308176
Recall: 0.6443556443556444
F1-score: 0.659846547314578
MCC: 0.3928794192026048
Confusion matrix: [[910 309]
 [356 645]]


We can see that, without changing the default parameters of each model, the Random Forest performs better than the Decision Tree. Still, the difference between the two is not very significant. So, the next step is tuning each model to get the best set of parameters for each model. If this doesn't improve the performance of each model, I will change the features used to train.

# Tuning each model

The function `evaluate_model_grid_search` uses the `GridSearchCV` class to find the best parameters for each model based on a score. I will use the `F1 score` metric because I want the classifier to classify both the negative and the positive classes well. 

In [ ]:
def evaluate_model_grid_search(model,parameters,score,train_data,train_labels,test_data,test_labels):

    estimator = GridSearchCV(model,parameters,scoring=score,refit=True,n_jobs=-1)
    estimator.fit(train_data,train_labels)

    best_params = estimator.best_params_
    print("Best Parameters:", best_params)

    predict = estimator.predict(test_data)
    evaluate_model(test_labels,predict)

## Decision Tree

In [23]:
parameters = {'max_depth': [None, 1,10,50,100],
              'min_samples_split': [2,10,50,100],
              'min_samples_leaf': [1,10,50,100],
              'max_features':["sqrt","log2",None],
              'criterion':['gini', 'entropy', 'log_loss']}

evaluate_model_grid_search(DecisionTreeClassifier(random_state=42),parameters,'f1',train_data,train_labels,test_data,test_labels)

Best Parameters: {'criterion': 'entropy', 'max_depth': None, 'max_features': 'sqrt', 'min_samples_leaf': 10, 'min_samples_split': 50}
Accuracy: 0.7319819819819819
Precision: 0.6726190476190477
Recall: 0.7902097902097902
F1-score: 0.7266881028938907
MCC: 0.4729211469926738
Confusion matrix: [[834 385]
 [210 791]]


## Random Forest

In [24]:
parameters = {'n_estimators':[5,10,100,500],
              'max_depth': [None, 1,10,50,100],
              'min_samples_split': [2,10,50,100],
              'min_samples_leaf': [1,10,50,100]}

evaluate_model_grid_search(RandomForestClassifier(random_state=42,n_jobs=-1),parameters,'f1',train_data,train_labels,test_data,test_labels)

Best Parameters: {'max_depth': None, 'min_samples_leaf': 100, 'min_samples_split': 2, 'n_estimators': 5}
Accuracy: 0.7509009009009009
Precision: 0.7328482328482329
Recall: 0.7042957042957043
F1-score: 0.7182883341823739
MCC: 0.49550642209404977
Confusion matrix: [[962 257]
 [296 705]]


# Adding a new features

With one more feature, we can increase the performance of our model. Looking at the column `'profile_background_color,'` we can see that most of the unpopular users use the color `F5F8FA` (Shadow), which is the background color. So, I will create a column called `'has_shadow'`, with a value of `1` for the users that use this color and `0` if they use other colors.

In [25]:
user_metadata_df['has_shadow'] = user_metadata_df['profile_background_color'].apply(lambda color: 1 if color == 'F5F8FA' else 0)
new_train_features = user_metadata_df[['geo_enabled','verified','default_profile','is_url_entities','statuses_count','has_shadow','popularity']]
new_train_features.head()

,geo_enabled,verified,default_profile,is_url_entities,statuses_count,has_shadow,popularity
0,1,0,0,0,4475,0,0
1,1,0,0,1,3922,0,0
2,0,0,1,0,11546,1,0
3,0,0,0,0,26609,0,1
4,1,0,0,0,519,0,0


In [26]:
features = new_train_features.drop(labels='popularity',axis=1)
labels = new_train_features['popularity']

In [ ]:
new_train_data,new_test_data,new_train_labels,new_test_labels = train_test_split(features,labels,train_size=0.8,test_size=0.2,random_state=42)

In [ ]:
scaler = StandardScaler()
new_train_data['statuses_count'] = scaler.fit_transform(new_train_data['statuses_count'].values.reshape(-1,1))
new_test_data['statuses_count'] = scaler.transform(new_test_data['statuses_count'].values.reshape(-1,1))

## Decision Tree

In [ ]:
model = DecisionTreeClassifier(random_state=42)
model.fit(new_train_data,new_train_labels)
predict = model.predict(new_test_data)
evaluate_model(new_test_labels,predict)

Accuracy: 0.7004504504504504
Precision: 0.6735537190082644
Recall: 0.6513486513486514
F1-score: 0.6622651091924835
MCC: 0.39345746857858094
Confusion matrix: [[903 316]
 [349 652]]


## Random Forest

In [ ]:
model = RandomForestClassifier(random_state=42,n_jobs=-1)
model.fit(new_train_data,new_train_labels)
predict = model.predict(new_test_data)
evaluate_model(new_test_labels,predict)

Accuracy: 0.7162162162162162
Precision: 0.6950578338590957
Recall: 0.6603396603396603
F1-score: 0.6772540983606556
MCC: 0.4247785627399015
Confusion matrix: [[929 290]
 [340 661]]


# Conclusions

Based on the presented resources, the chosen features are relevant to a popular profile on Twitter. With these features and some parameters tuning, I created a model with around `75%` accuracy, which is a good performance. There are more factors that I have not considered that influence a user's popularity, like the topic of his profile or other factors. However, with this data, it was possible to find some of these factors and predict with some success if a profile is popular or not.  